# 1. Import library

In [61]:
# python native
import os
import json
import random
import datetime
from functools import partial

# external library
import cv2
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from sklearn.model_selection import GroupKFold
import albumentations as A

# torch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models
from torchvision.models.segmentation import deeplabv3_resnet101

# visualization
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# 2. Load data

In [62]:
# 기학습 모델 사용하여 test image의 capitate infernce data load
df=pd.read_csv('./output_unet++_diceiou.csv')

In [63]:
# Capitate 분리
df_capitate = df[df['class']=='Capitate']
df_capitate

,image_name,class,rle
21,image1661319116107.png,Capitate,2954193 3 2956237 12 2958282 19 2960328 23 296...
50,image1661319145363.png,Capitate,2806762 5 2808808 12 2810853 19 2812895 26 281...
79,image1661319356239.png,Capitate,2741231 8 2743277 12 2745321 19 2747363 28 274...
108,image1661319390106.png,Capitate,2714523 22 2716570 26 2718616 31 2720661 40 27...
137,image1661320372752.png,Capitate,2767876 4 2769902 30 2771939 43 2773982 50 277...
...,...,...,...
8228,image1664154624841.png,Capitate,2452479 13 2454524 18 2456566 33 2458599 51 24...
8257,image1667354218823.png,Capitate,2790428 10 2790440 12 2792472 33 2794515 41 27...
8286,image1667354234602.png,Capitate,2759653 3 2761694 25 2763739 34 2765786 44 276...
8315,image1667354405140.png,Capitate,2687982 2 2690013 13 2690027 8 2692024 5 26920...


# 3. Decode RLE & Get coordinate

In [64]:
# Decode rle to mask
def decode_rle_to_mask(rle, height, width):
    s = rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(height * width, dtype=np.uint8)
    
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    
    return img.reshape(height, width)

In [65]:
def capitate_mask_coor(dataframe, imageidx, height, width):
    capitate_mask = decode_rle_to_mask(dataframe.iloc[imageidx]['rle'], height, width)
    file_name = dataframe.iloc[imageidx]['image_name']
    y_coor, x_coor = np.where(capitate_mask == 1)
    x_mean, y_mean = int(np.mean(x_coor)), int(np.mean(y_coor))
    
    return file_name, (x_mean, y_mean)

'''
imageidx: sample을 확인할 image index
width, height = 확인용 원본 image size
'''
imageidx = 0
width, height = 2048, 2048
filename, coor = capitate_mask_coor(df_capitate, imageidx, height, width)

print(f'File name: {filename}')
print(f'Coor: {coor}')

File name: image1661319116107.png
Coor: (953, 1529)


# 4. Check

In [66]:
# Test image root
test_image_root = '/data/ephemeral/home/data/test/DCM'

In [67]:
# Test image ID/filename 저장
sample_path = '/data/ephemeral/home/data/test/DCM/ID040/image1661319116107.png'
sample_path = 'image1661319116107.png'

pngs = [
    os.path.relpath(os.path.join(root, fname), start=test_image_root)
    for root, dirs, files in os.walk(test_image_root)
    for fname in files
    if os.path.splitext(fname)[1].lower() == '.png'
]

pngs = sorted(pngs)

In [68]:
# Make crop_coor
def crop_coordinate(coordinate):
    x, y = coordinate
    x_min = x - 256
    y_min = y - 256
    x_max = x + 256
    y_max = y + 256
    
    return x_min, y_min, x_max, y_max

rectangle_x, rectangle_y = crop_coordinate(coor)[0], crop_coordinate(coor)[1]

print(rectangle_x, rectangle_y)

697 1273


In [69]:
if filename == pngs[imageidx].split('/')[1]:
    print('Coordinate is matched image_file')
else:
    print('Check the middle_coordinate and image_file')

# crop size & load image
crop_width, crop_height = 512, 512
image = cv2.imread(os.path.join(test_image_root, pngs[imageidx]))

Coordinate is matched image_file


# 5. Make all crop_coordinate

In [71]:
crop_data = []

def make_crop_coor(dataframe, imageidx, height, width):
    filename, coor = capitate_mask_coor(dataframe=dataframe, 
                                        imageidx=imageidx, 
                                        height=height, 
                                        width=width)
    x_min, y_min, x_max, y_max = crop_coordinate(coor)

    return filename, x_min, y_min, x_max, y_max

height, width = 2048, 2048

for idx in range(len(df_capitate)):
    filename, x_min, y_min, x_max, y_max = make_crop_coor(df_capitate, idx, height, width)

    crop_data.append([filename, x_min, y_min, x_max, y_max])

df_crop = pd.DataFrame(crop_data, columns=['filename', 'x_min', 'y_min', 'x_max', 'y_max'])

df_crop
    

,filename,x_min,y_min,x_max,y_max
0,image1661319116107.png,697,1273,1209,1785
1,image1661319145363.png,770,1203,1282,1715
2,image1661319356239.png,726,1167,1238,1679
3,image1661319390106.png,703,1151,1215,1663
4,image1661320372752.png,737,1164,1249,1676
...,...,...,...,...,...
283,image1664154624841.png,785,1004,1297,1516
284,image1667354218823.png,776,1189,1288,1701
285,image1667354234602.png,792,1176,1304,1688
286,image1667354405140.png,707,1146,1219,1658


In [72]:
paired_list = list(zip(df_crop['x_min'], df_crop['y_min']))

print(paired_list)

[(697, 1273), (770, 1203), (726, 1167), (703, 1151), (737, 1164), (771, 1180), (719, 1257), (770, 1264), (748, 1261), (701, 1193), (732, 1141), (727, 1151), (739, 1178), (735, 1124), (800, 1232), (718, 1202), (727, 1201), (717, 1212), (821, 1123), (707, 1129), (771, 1186), (693, 1164), (797, 1176), (721, 1202), (755, 1075), (768, 1113), (743, 1165), (789, 1202), (778, 1255), (800, 1213), (793, 1146), (758, 1155), (722, 1221), (754, 1196), (761, 1123), (763, 1169), (719, 1213), (778, 1230), (734, 1118), (725, 1118), (765, 1156), (806, 1139), (718, 1195), (746, 1156), (730, 1113), (747, 1133), (745, 1186), (759, 1136), (763, 1173), (731, 1159), (745, 1128), (758, 1116), (732, 1157), (686, 1142), (768, 1224), (716, 1204), (771, 1132), (751, 1128), (735, 1100), (733, 1182), (731, 1087), (717, 1102), (718, 1374), (668, 1213), (816, 1182), (707, 1169), (796, 1104), (682, 1116), (740, 1196), (782, 1152), (786, 1165), (736, 1117), (721, 1146), (749, 1175), (792, 1127), (748, 1127), (729, 1183)

In [73]:
wdf=pd.read_csv("./output_unet++_diceiou_wrist.csv")


In [74]:
def encode_mask_to_rle(mask):
    """
    mask: numpy array binary mask
    1 - mask
    0 - background
    Returns encoded run length
    """
    pixels = mask.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return " ".join(str(x) for x in runs)

# RLE로 인코딩된 결과를 mask map으로 복원합니다. (RLE -> mask map)
def decode_rle_to_mask_2(rle, height, width,x,y):
    s = str(rle).split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(height * width, dtype=np.uint8)
    img=img.reshape(height, width)
    for lo, hi in zip(starts, ends):
        for n in range(lo,hi):
            img[(n//512)+y][(n%512)+x] = 1
    
    return img

In [75]:
# 29 * 288
# df length를 29개씩 끊어서 처리
cnt=0
for i in tqdm(range(0, len(wdf), 29)):#한 이미지씩 
    # class number
    x,y=paired_list[cnt]
    for j in range(19,27): # 한 이미지의 한 클래스씩
        result = np.zeros((2048, 2048), dtype=int)# 이미지 빈칸
        rle = wdf.iloc[i+j].rle #i는 시작점 j는 클래스 0+0 =finger-1 .rle = 한 열 
        mask = decode_rle_to_mask_2(rle, 2048, 2048,x,y) # 01 matrix 마스크픽셀 식으로 바꾸기
        # calculate with threshold
        wdf.rle[i+j] = encode_mask_to_rle(mask) # 다시 rle로 바꿈
    cnt +=1
    


  0%|          | 0/288 [00:00<?, ?it/s]

/tmp/ipykernel_1823253/1930068302.py:12: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  wdf.rle[i+j] = encode_mask_to_rle(mask) # 다시 rle로 바꿈


In [76]:
wdf.to_csv("wrist.csv", index=False)